In [ ]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
%cd drive/MyDrive/semantic_mapping_project/

/content/drive/MyDrive/semantic_mapping_project


In [2]:
import glob
import json
from tqdm import tqdm
import pandas as pd

tt_data = {
          'house_id': [],
          'file_name': [],
          'frames_dir': [],
          'target_obj': [],
          'action': [],
          'human_intention': [],
          'gpt3_intension_wd': []
      }

for folder_name in tqdm(glob.glob("./"+"**_expert_annotations",recursive=True)):
  for file_name in tqdm(glob.glob(folder_name+"/"+"forTraining/house_**_**.json",recursive=True)):
    with open(file_name, 'r') as f:
      data = json.load(f)
    target_object_type = data['target_object_type']
    counter = 0
    for ac_id in tqdm(range(len(data['actions']))):
    
      tt_data['house_id'].append(data['house_id'])
      tt_data['file_name'].append(file_name.strip('.json'))
      tt_data['frames_dir'].append(data['frames_dir'] + "frame_"+ str(counter))
      tt_data['target_obj'].append(data['target_object_type'])
      tt_data['action'].append(data['actions'][ac_id]['action'])
      tt_data['human_intention'].append(data['actions'][ac_id]['intentions'])
      tt_data['gpt3_intension_wd'].append(data['actions'][ac_id]['gpt3_intention_wd'])

      counter += 1

df = pd.DataFrame(data=tt_data)
df['frame_count'] = df['frames_dir'].apply(lambda x: int(x.split('_')[-1]))
df['Without_direction'] = df['file_name'].apply(lambda x: '_' == x[-1])

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 99/99 [00:00<00:00, 186455.36it/s]

  6%|▌         | 1/18 [00:00<00:06,  2.73it/s]

100%|██████████| 72/72 [00:00<00:00, 177536.68it/s]

 11%|█         | 2/18 [00:00<00:06,  2.54it/s]

100%|██████████| 63/63 [00:00<00:00, 173728.57it/s]

 17%|█▋        | 3/18 [00:01<00:05,  2.58it/s]

100%|██████████| 59/59 [00:00<00:00, 156030.22it/s]

 22%|██▏       | 4/18 [00:01<00:05,  2.57it/s]

100%|██████████| 55/55 [00:00<00:00, 185439.49it/s]

 28%|██▊       | 5/18 [00:01<00:04,  2.62it/s]

100%|██████████| 58/58 [00:00<00:00, 24285.68it/s]

 33%|███▎      | 6/18 [00:02<00:04,  2.57it/s]

100%|██████████| 64/64 [00:00<00:00, 179916.53it/s]

 39%|███▉      | 7/18 [00:02<00:04,  2.30it/s]

100%|██████████| 29/29 [00:00<00:00, 122987.68it/s]

 44%|████▍     | 8/18 [00:03<00:04,  2.22it/s]

100%|██████████| 74/74 [00:00<00:00, 163615.44it/s]

 50%|█████     | 9/18 [00:03<00:03,  2.26it/s]

100%|██████████| 121/121 [00:00<00:00, 29256.40it/s]

In [3]:
df['target_obj'].unique()

array(['Book', 'Pillow', 'SprayBottle', 'Mug', 'Apple', 'Bowl',
       'RemoteControl', 'Laptop', 'Pen', 'AlarmClock'], dtype=object)

In [ ]:
df = df[(df['action'] != 'PickUpObject') & (df['action'] != 'OpenObject') & (df['action'] != 'end') & (df['action'] != 'LookDown') & (df['action'] != 'LookUp')]

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

le = LabelEncoder()

df['folder_name'] = df['file_name'].apply(lambda x: x.split('/')[1] + "/")
df['image_path'] = df['folder_name'] + df['frames_dir'].apply(lambda x : "/".join(x.split('/')[1:]))
df['label'] = le.fit_transform(list(df['action']))

In [ ]:
le.classes_

array(['MoveAhead', 'MoveBack', 'MoveLeft', 'MoveRight', 'RotateLeft',
       'RotateRight'], dtype='<U11')

In [ ]:
df['label'].value_counts()

0    6294
5     684
3     613
4     590
2     515
1      22
Name: label, dtype: int64

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
train_col = ['gpt3_intension_wd', 'image_path']
target_col = ['action']

In [ ]:

from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=0.2, random_state= 42)

In [ ]:
import torchvision
from torchvision import models, transforms
import numpy as np
import transformers

import torch.nn as nn

import torch
import pandas as pd

from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

import torch.nn.functional as F
from transformers import BertModel, BertTokenizer
import random
import time
import os
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter


In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
class TextEncoder(nn.Module):

    def __init__(self, text_fc2_out=32, text_fc1_out=128, dropout_p=0.4, fine_tune_module=False):
        super(TextEncoder, self).__init__()
        self.fine_tune_module = fine_tune_module

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained(
                    'bert-base-uncased',
                    return_dict=True)

        self.text_enc_fc1 = torch.nn.Linear(768, text_fc1_out)
        self.text_enc_fc2 = torch.nn.Linear(text_fc1_out, text_fc2_out)

        self.dropout = nn.Dropout(dropout_p)

        self.fine_tune()
        
    def forward(self, input_ids, attention_mask):
        
        # Feed input to BERT
        out = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        ## odict_keys(['last_hidden_state', 'pooler_output', 'attentions'])
        ## last_hidden_state (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)) 

        x = self.dropout(
            torch.nn.functional.relu(
                self.text_enc_fc1(out['pooler_output']))
        )    

        x = self.dropout(
            torch.nn.functional.relu(
                self.text_enc_fc2(x))
        ) 
        
        return x
    
    def fine_tune(self):
        """
        keep the weights fixed or not  
        """
        for p in self.bert.parameters():
            p.requires_grad = self.fine_tune_module
            

            
class VisionEncoder(nn.Module):
    """Visual Feature extraction
    """
    def __init__(self, img_fc1_out=128, img_fc2_out=32, dropout_p=0.4, fine_tune_module=False):
        """
        """
        super(VisionEncoder, self).__init__()
        
        self.fine_tune_module = fine_tune_module

        
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, img_fc1_out)
  
        self.vis_encoder = self.resnet

        self.vis_enc_fc2 = torch.nn.Linear(img_fc1_out, img_fc2_out)

        self.dropout = nn.Dropout(dropout_p)

        self.fine_tune()
        

        
    def forward(self, images):
        """
        Forward propagation.
        :param images: images, a tensor of dimensions (batch_size, 3, image_size, image_size)
        :return: encoded images
        """

        x = self.vis_encoder(images)

        x = self.dropout(
            torch.nn.functional.relu(
                self.vis_enc_fc2(x))
        )

        return x
    
    def fine_tune(self):
        """
        Allow or prevent the computation of gradients for convolutional blocks 2 through 4 of the encoder.
        :param fine_tune: Allow?
        """
        for p in self.resnet.parameters():
            p.requires_grad = False

        # If fine-tuning, only fine-tune convolutional blocks 2 through 4
        for c in list(self.resnet.children())[5:]:
            for p in c.parameters():
                p.requires_grad = self.fine_tune_module
                
class LanguageAndVisionConcat(torch.nn.Module):
    def __init__(
        self,
        model_params
        
    ):
        super(LanguageAndVisionConcat, self).__init__()
        
        self.text_encoder = TextEncoder(model_params['text_fc2_out'], model_params['text_fc1_out'], model_params['dropout_p'], model_params['fine_tune_text_module'])
        self.vision_encode = VisionEncoder(model_params['img_fc1_out'], model_params['img_fc2_out'], model_params['dropout_p'], model_params['fine_tune_vis_module'])

        self.fusion = torch.nn.Linear(
            in_features=(model_params['text_fc2_out'] + model_params['img_fc2_out']), 
            out_features=model_params['fusion_output_size']
        )
        self.fc = torch.nn.Linear(
            in_features=model_params['fusion_output_size'], 
            out_features=6
        )
        self.dropout = torch.nn.Dropout(model_params['dropout_p'])


    def forward(self, text, image, label=None):

        ## Pass the text input to Bert encoder 
        text_features = self.text_encoder(text[0], text[1])

        ## Pass the image input 
        image_features = self.vision_encode(image)

        ## concatenating Image and text 
        combined_features = torch.cat(
            [text_features, image_features], dim=1
        )        

        combined_features = self.dropout(combined_features)

        fused = self.dropout(
            torch.nn.functional.relu(
            self.fusion(combined_features)
            )
        )

        prediction = self.fc(fused)

        return prediction      

In [ ]:

class RobotIntensionDataset(Dataset):

    def __init__(self, df, root_dir, image_transform, tokenizer, MAX_LEN):
        """
        Args:
            csv_file (string): Path to the csv file with text and img name.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_data = df
        self.root_dir = root_dir
        self.image_transform = image_transform
        self.tokenizer_bert = tokenizer
        self.MAX_LEN = MAX_LEN

    def __len__(self):
        return self.csv_data.shape[0]
    
    def pre_processing_BERT(self, sent):
        # Create empty lists to store outputs
        input_ids = []
        attention_mask = []
        
        encoded_sent = self.tokenizer_bert.encode_plus(
            text= sent.lower(),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length= self.MAX_LEN,                  # Max length to truncate/pad
            padding='max_length',         # Pad sentence to max length
            # return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )
        
        input_ids = encoded_sent.get('input_ids')
        attention_mask = encoded_sent.get('attention_mask')
        
        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        
        return input_ids, attention_mask
     
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = self.root_dir + list(self.csv_data['image_path'])[idx]+'.jpg'
        
        image = Image.open(img_name).convert("RGB")
        
        image = self.image_transform(image)
        
        text = list(self.csv_data['gpt3_intension_wd'])[idx]
        
        tensor_input_id, tensor_input_mask = self.pre_processing_BERT(text)

        label = list(self.csv_data['label'])[idx]
        label = torch.tensor(label)

        sample = {'image': image, 'BERT_ip': [tensor_input_id, tensor_input_mask], 'label':label}

        return sample

In [ ]:
image_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(224, 224)),
        torchvision.transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

MAX_LEN = 500
NUM_OF_CLASSES = 6
root_dir = "./"

# Run function `preprocessing_for_bert` on the dataset
transformed_dataset_train = RobotIntensionDataset(df_train, root_dir, image_transform, tokenizer, MAX_LEN)

transformed_dataset_val = RobotIntensionDataset(df_val, root_dir, image_transform, tokenizer, MAX_LEN)

train_dataloader = DataLoader(transformed_dataset_train, batch_size=12,
                        shuffle=True, num_workers=0)

val_dataloader = DataLoader(transformed_dataset_val, batch_size=12,
                        shuffle=True, num_workers=0)

parameter_dict_model={
    'text_fc2_out': 32, 
    'text_fc1_out': 128, 
    'dropout_p': 0.4, 
    'fine_tune_text_module': True,
    'img_fc1_out': 128, 
    'img_fc2_out': 32, 
    'dropout_p': 0.4, 
    'fine_tune_vis_module': True,
    'fusion_output_size': 35
}

parameter_dict_opt={'l_r': 3e-5,
                    'eps': 1e-8
                    }


EPOCHS = 50

final_model = LanguageAndVisionConcat(parameter_dict_model)

final_model = final_model.to(device) 

# Create the optimizer
optimizer = AdamW(final_model.parameters(),
                  lr=parameter_dict_opt['l_r'],
                  eps=parameter_dict_opt['eps'])

# Total number of training steps
total_steps = len(train_dataloader) * EPOCHS

# Set up the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # Default value
                                            num_training_steps=total_steps)


## Instantiate the tensorboard summary writer
writer = SummaryWriter('runs/multi_att_exp3')

loss_fn = nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be remov

In [ ]:
def train(model, loss_fn, optimizer, scheduler, train_dataloader, val_dataloader=None, epochs=4, evaluation=False, device='cpu', param_dict_model=None, param_dict_opt=None, save_best=False, file_path='./saved_models/best_model.pt', writer=None):
    """Train the BertClassifier model.
    """
    # Start training loop
    best_acc_val = 0
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            img_ip , text_ip, label = batch["image"], batch["BERT_ip"], batch['label']
            
            b_input_ids, b_attn_mask = tuple(t.to(device) for t in text_ip)
            
            imgs_ip = img_ip.to(device)
            
            b_labels = label.to(device)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(text=[b_input_ids, b_attn_mask], image=imgs_ip, label=b_labels)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()
            
#             break 
            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")
                
                ## Write onto tensorboard
                if writer != None:
                    writer.add_scalar('Training Loss', (batch_loss / batch_counts), epoch_i*len(train_dataloader)+step)
                
                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, loss_fn, val_dataloader, device)            
            
            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
            
            ## Write onto tensorboard
            if writer != None:
                writer.add_scalar('Validation Loss', val_loss, epoch_i+1)
                writer.add_scalar('Validation Accuracy', val_accuracy, epoch_i+1)
            
            # Save the best model
            if save_best: 
                if val_accuracy > best_acc_val:
                    best_acc_val = val_accuracy
                    torch.save({
                                'epoch': epoch_i+1,
                                'model_params': param_dict_model,
                                'opt_params': param_dict_opt,
                                'model_state_dict': model.state_dict(),
                                'opt_state_dict': optimizer.state_dict(),
                                'sch_state_dict': scheduler.state_dict()
                               }, file_path)
                    
        print("\n")
    
    print("Training complete!")
    
    
    
def evaluate(model, loss_fn, val_dataloader, device):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        img_ip , text_ip, label = batch["image"], batch["BERT_ip"], batch['label']
            
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in text_ip)

        imgs_ip = img_ip.to(device)

        b_labels = label.to(device)

        # Compute logits
        with torch.no_grad():
            logits = model(text=[b_input_ids, b_attn_mask], image=imgs_ip, label=b_labels)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
train(model=final_model, loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler, train_dataloader=train_dataloader, val_dataloader=val_dataloader, epochs=150, evaluation=True, device=device, param_dict_model=parameter_dict_model, param_dict_opt=parameter_dict_opt, save_best=True, file_path='./saved_models/best_model.pt', writer=writer)


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.692396   |     -      |     -     |   96.16  
   1    |   40    |   1.563202   |     -      |     -     |   92.65  
   1    |   60    |   1.521665   |     -      |     -     |   87.39  
   1    |   80    |   1.419043   |     -      |     -     |   87.42  
   1    |   100   |   1.361218   |     -      |     -     |   86.92  
   1    |   120   |   1.282339   |     -      |     -     |   80.81  
   1    |   140   |   1.387449   |     -      |     -     |   85.20  
   1    |   160   |   1.345350   |     -      |     -     |   83.23  
   1    |   180   |   1.305800   |     -      |     -     |   81.83  
   1    |   200   |   1.205654   |     -      |     -     |   78.50  
   1    |   220   |   1.252229   |     -      |     -     |   81.26  
   1    |   240   |   1.128689   |     -      |     -     |   92.07  


FileNotFoundError: ignored